In [1]:
%load_ext autoreload
%autoreload 2

In [91]:
from food.tools import *
from food.paths import *

import pandas as pd
import numpy as np
import requests
import telebot
from food.psql import *

# !nbdev_build_lib
from food.classify_image import *

from telebot import types

import datetime
import pytz
from tzwhere import tzwhere


debug = True

dishes = 'dishes_test'  if debug else 'dishes'
users = 'users_test'  if debug else 'users'

token = "5091011572:AAG4NfkC_zZjcsaAFkwLm4ZXOvhEqyLpQhY" if debug else '5203882708:AAG3G_Y2oZSr-rMG2zoffDVtj3d0KkOFSts'



m = None

bot = telebot.TeleBot(token)
bot.dish = None


@bot.message_handler(commands=['start'])
def welcome(message):
	markup = types.ReplyKeyboardMarkup(row_width=1,resize_keyboard=True)
	markup.add('/help')
	bot.reply_to(message, "Counting calories as easy as taking pictures. Just capture everything before you eat it", reply_markup=markup)
	bot.reply_to(message, "Now send a photo of your meal to try", reply_markup=markup)

@bot.message_handler(content_types=['location'])
def location(message):
	bot.send_chat_action(message.chat.id, 'typing')
	tz = tzwhere.tzwhere().tzNameAt(message.location.latitude, message.location.longitude)

	df = pd.DataFrame([[message.from_user.id,
              message.location.latitude,
              message.location.longitude,
              tz]],
              columns = ['id','lat','lon','timezone'])

	insert_ignore(df,users,update = True, update_cols = ['lat','lon','timezone'], engine = engine,unique_cols=['id'])
	bot.reply_to(message, f"your timezone is set to {tz}")



@bot.message_handler(commands=['help'])
def help(message):
	bot.reply_to(message, "join our support group https://t.me/+nIBkPkw3vpM0NDJi")

@bot.message_handler(commands=['cancel'])
def send_cancel(message):
	markup = types.ReplyKeyboardRemove(selective=False)
	bot.reply_to(message, f"cancalled", reply_markup=markup)
	if hasattr(bot,'dish'): del bot.dish
	if hasattr(bot,'measures'): del bot.measures
	if hasattr(bot,'measurement'): del bot.measurement
	if hasattr(bot,'label'): del bot.label

@bot.message_handler(content_types=['photo'])
def calssify_image(message):

	bot.send_chat_action(message.chat.id, 'typing')
	
	image_url = bot.get_file_url(message.photo[3].file_id)
	bot.dish = search_image(url=image_url)
	bot.dish['image_url'] = image_url
	bot.dish['user_id']= message.from_user.id

	plot_numtients = bot.dish[['energy','protein','carb','fat']].reset_index(drop=True)
	plot_numtients.index = ['']


	bot.label = bot.dish['description'].iloc[0]

	bot.reply_to(message, bot.label)
	bot.reply_to(message, plot_numtients.to_string())

	bot.measures = pd.read_sql(f"select portion_description,gram_weight from portions where food_id = {bot.dish['id'].iloc[0]}",engine)
	bot.measures_cleaned = bot.measures['portion_description'].str.replace('1 ','').tolist()+['gram']

	markup = types.ReplyKeyboardMarkup(row_width=2)
	cancel = types.KeyboardButton('/cancel')
	[markup.add(p) for p in bot.measures_cleaned]
	markup.add(cancel)

	bot.send_message(message.chat.id, "Choose the unit to measure the weight of you dish", reply_markup=markup)


@bot.message_handler(content_types=['text']) #regexp="[0-9]+"
def handle_text(message):
	global m
	m = message

	bot.message = message
	try:     bot.number = int(message.text)
	except:  bot.number = None

	if not bot.number and hasattr(bot,'label'):
		

		if len(set(bot.measures_cleaned) & set([message.text])) >0:
			bot.measurement = message.text
			markup = types.ReplyKeyboardMarkup(row_width=2)
			if bot.measurement == 'gram': [markup.add(str(p)) for p in range(20,400,20)]
			else:[markup.add(str(p)) for p in range(1,5)]
			cancel = types.KeyboardButton('/cancel')
			markup.add(cancel)
			bot.reply_to(message, f"select number of {bot.measurement}s you are going to eat", reply_markup=markup)

		else:
			bot.reply_to(message, f"please choose a unit from the list")



	

	elif bot.number and hasattr(bot,'label'): 
		if bot.measurement == 'gram': grams = bot.number
		else:grams = bot.measures[bot.measures['portion_description'].str.contains(bot.measurement, regex=False)]['gram_weight'].iloc[0]*bot.number
		bot.dish[['energy','protein','carb','fat']] = bot.dish[['energy','protein','carb','fat']]/100*grams
		bot.dish['grams']=grams
		bot.dish['measure_selected'] = message.text
		bot.dish = bot.dish.rename(columns = {'id':'food_id'})
		bot.dish['timestamp']=pd.Timestamp.utcnow()

		bot.dish.to_sql(dishes,engine,if_exists='append',index=False)

	
		markup = types.ReplyKeyboardRemove(selective=False)

		today_consumed = pd.read_sql(f"select energy,timestamp from {dishes} where user_id = {message.from_user.id} and timestamp > now() - interval '24 hours';",engine).set_index("timestamp")
		user_tz = engine.execute(f'select timezone from {users} where id={message.from_user.id}').first()
		user_tz = user_tz[0] if user_tz else 'UTC'
		today_consumed = today_consumed.tz_convert(user_tz)
		if user_tz == 'UTC': bot.reply_to(message, "Please send your location to that we know your local time", reply_markup=markup)
		now = pd.Timestamp.now(tz = user_tz)
		today_consumed = today_consumed.reset_index()	
		this_morning = pd.Timestamp(year = now.year,month = now.month,day = now.day,hour = 3,tz = user_tz)
		today_consumed = today_consumed[today_consumed['timestamp'] > pd.Timestamp(this_morning)]['energy'].sum()


		bot.reply_to(message, f" {bot.number} {bot.measurement}s  of {bot.label} added", reply_markup=markup)
		bot.reply_to(message, f"You have consumed {round(today_consumed)} kcall today", reply_markup=markup)

		del bot.dish
		del bot.measures
		del bot.measurement
		del bot.label

	elif not hasattr(bot,'label'):

		bot.reply_to(message, f"Please take a photo of your dish")




	

bot.infinity_polling()

In [52]:
message = m 

In [88]:
today_consumed = pd.read_sql(f"select energy,timestamp from {dishes} where user_id = {message.from_user.id} and timestamp > now() - interval '24 hours';",engine).set_index("timestamp")
user_tz = engine.execute(f'select timezone from {users} where id={message.from_user.id}').first()
user_tz = user_tz[0] if user_tz else 'UTC'
today_consumed = today_consumed.tz_convert(user_tz)
if user_tz == 'UTC': bot.reply_to(message, "Please send your location to that we know your local time", reply_markup=markup)
now = pd.Timestamp.now(tz = user_tz)
today_consumed = today_consumed.reset_index()	
this_morning = pd.Timestamp(year = now.year,month = now.month,day = now.day,hour = 3,tz = user_tz)
today_consumed = today_consumed[today_consumed['timestamp'] > pd.Timestamp(this_morning)]['energy'].sum()



In [89]:
today_consumed

307.66

In [87]:
user_tz

'UTC'

In [46]:
user_tz = engine.execute(f'select timezone from {users} where id={message.from_user.id}').first()

In [49]:
user_tz

('Europe/Lisbon',)

In [50]:
pd.Timestamp.now(tz = user_tz[0])

Timestamp('2022-04-06 09:50:03.400990+0100', tz='Europe/Lisbon')

In [12]:
message = m

In [13]:
message.from_user

In [16]:
today_consumed = pd.read_sql(f"select energy,timestamp from {dishes} where user_id = {message.from_user.id} and timestamp > now() - interval '24 hours';",engine).set_index("timestamp")


In [17]:
today_consumed

,energy
timestamp,
2022-04-05 16:26:11.364469+00:00,375.00
2022-04-05 16:58:46.104406+00:00,1.30
2022-04-05 16:59:21.881218+00:00,113.68
2022-04-05 19:38:07.085790+00:00,260.00
2022-04-05 19:46:51.486692+00:00,130.00
2022-04-05 19:47:19.810591+00:00,325.00
2022-04-05 19:49:00.238328+00:00,325.00
2022-04-05 19:53:44.223082+00:00,130.00
2022-04-05 19:54:46.417506+00:00,85.80


In [18]:

user_tz = engine.execute(f'select timezone from {users} where id={message.from_user.id}').first()


In [19]:
user_tz

('Europe/Lisbon',)

In [23]:

if user_tz: today_consumed = today_consumed.tz_convert(user_tz[0])


else: bot.reply_to(message, "Please send your location to that we know your local time", reply_markup=markup)
today_consumed = today_consumed.reset_index()	


In [24]:
today_consumed

,timestamp,energy
0,2022-04-05 17:26:11.364469+01:00,375.00
1,2022-04-05 17:58:46.104406+01:00,1.30
2,2022-04-05 17:59:21.881218+01:00,113.68
3,2022-04-05 20:38:07.085790+01:00,260.00
4,2022-04-05 20:46:51.486692+01:00,130.00
5,2022-04-05 20:47:19.810591+01:00,325.00
6,2022-04-05 20:49:00.238328+01:00,325.00
7,2022-04-05 20:53:44.223082+01:00,130.00
8,2022-04-05 20:54:46.417506+01:00,85.80
9,2022-04-05 20:56:32.351239+01:00,130.00


In [28]:
pd.Timestamp.utcnow().date()

datetime.date(2022, 4, 6)

In [33]:
now = pd.Timestamp.utcnow()
pd.Timestamp(year = now.year,month = now.month,day = now.day,hour = 3)

In [25]:
today_consumed[today_consumed['timestamp'] > pd.Timestamp]

,timestamp,energy
0,2022-04-05 17:26:11.364469+01:00,375.00
1,2022-04-05 17:58:46.104406+01:00,1.30
2,2022-04-05 17:59:21.881218+01:00,113.68
3,2022-04-05 20:38:07.085790+01:00,260.00
4,2022-04-05 20:46:51.486692+01:00,130.00
5,2022-04-05 20:47:19.810591+01:00,325.00
6,2022-04-05 20:49:00.238328+01:00,325.00
7,2022-04-05 20:53:44.223082+01:00,130.00
8,2022-04-05 20:54:46.417506+01:00,85.80
9,2022-04-05 20:56:32.351239+01:00,130.00


In [ ]:

today_consumed = today_consumed[today_consumed['timestamp'].dt.time > datetime.time(3,0)]['energy'].sum()


In [15]:
today_consumed

4510.279999999999